In [ ]:
import json
import pickle
from bisect import bisect_right
from collections import defaultdict
from pathlib import Path

import networkx as nx
import numpy as np
import pandas as pd
from bitarray.util import zeros
from intervaltree import IntervalTree
from matplotlib import pyplot as plt
from networkx.readwrite.json_graph.node_link import node_link_graph
from tqdm.notebook import tqdm

from dscent.time_sequence import trim_before, trim_after
from input.iterator import GraphEdgeIterator

In [ ]:
raw_df = pd.read_csv("may-run-omega0050_cycles.csv", delimiter=";")
raw_df["candidates"] = raw_df["candidates"].apply(lambda x: set(json.loads(x)))
raw_df["bundled_cycle"] = raw_df["bundled_cycle"].apply(lambda x: (node_link_graph(json.loads(x), edges="edges")))
raw_df["vertices"] = raw_df["bundled_cycle"].apply(lambda x: set(x.nodes()))
for index, cycle in tqdm(raw_df["bundled_cycle"].items(), total=len(raw_df), desc="Processing cycles", unit_scale=True):
    first, *_, last = sorted(cycle.edges(keys=True), key=lambda x: x[2])
    raw_df.loc[index, "begin"] = first[2]
    raw_df.loc[index, "end"] = last[2]
raw_df[["begin", "end"]] = raw_df[["begin", "end"]].astype(int)

In [ ]:
# Build interval tree with row index as data
intervals = IntervalTree()
for idx, row in tqdm(raw_df.iterrows(), total=len(raw_df), desc="Building intervals", unit_scale=True):
    intervals[row["begin"]: row["end"] + 1] = idx  # End is inclusive

In [ ]:
chunk_size = 10_000
sorted_keys, sorted_values = zip(*intervals.boundary_table.items())

aggregated_keys = []
aggregated_values = []

for i in range(0, len(sorted_keys), chunk_size):
    chunk_keys = sorted_keys[i:i + chunk_size]
    chunk_values = sorted_values[i:i + chunk_size]

    # Use the midpoint key as a representative key
    aggregated_keys.append(np.mean(chunk_keys))
    aggregated_values.append(sum(chunk_values))

# Plotting
plt.plot(aggregated_keys, aggregated_values)
plt.yscale("log")
plt.xlabel("Mean key of chunk")
plt.ylabel("Sum of values in chunk")
plt.title("Aggregated Boundary Table (100-key bins)")
plt.show()

In [ ]:
    # Step 1: Create a mapping from vertex to bit position
all_vertices = set.union(*raw_df["vertices"])
vertex_to_index = {v: i for i, v in enumerate(sorted(all_vertices))}
num_bits = len(vertex_to_index)

# Step 2: Convert each vertex set to a bitarray
def set_to_bitarray(vertex_set):
    ba = zeros(num_bits)
    for v in vertex_set:
        ba[vertex_to_index[v]] = 1
    return ba

vertices_bitarrays = []
for i, vertices in tqdm(raw_df["vertices"].items(), total=len(raw_df), unit_scale=True):
    vertices_bitarrays.append(set_to_bitarray(vertices))

In [ ]:
# Step 3: Proceed with your logic using bitarrays
edge_list = []
sorted_df = raw_df.sort_values(["begin", "end"], ascending=[True, False])
grouped_by_begin = sorted_df.groupby("begin")

pbar = tqdm(total=len(raw_df[["begin", "end"]].drop_duplicates()), desc="Building meta graph", unit_scale=True, smoothing=1)

for begin_i, begin_group in grouped_by_begin:
    max_end = begin_group["end"].iloc[0]

    current_intervals = sorted(
        intervals[begin_i: max_end + 1],
        key=lambda iv: (iv.begin, -iv.end)
    )
    slice_index = len(current_intervals)

    for end_i, group in begin_group.groupby("end"):
        new_slice_index = bisect_right([iv.begin for iv in current_intervals], end_i)
        if new_slice_index < slice_index:
            slice_index = new_slice_index
            current_intervals = current_intervals[: new_slice_index]

        overlap_indices = {iv.data for iv in current_intervals}

        for i in group.index:
            vertices_bitarray_i = vertices_bitarrays[i]
            for j in overlap_indices:
                if j > i and (vertices_bitarray_i & vertices_bitarrays[j]).any():
                    edge_list.append((i, j))
        pbar.update(1)

meta_graph = nx.Graph()
meta_graph.add_edges_from(edge_list)

In [ ]:
len(edge_list), num_bits ** 2

In [ ]:
def get_index(address_string):
    for chunk in pd.read_csv("input/node_ids.txt", header=None, chunksize=10_000):
        for eth_index, eth_address in chunk.iloc[:, 0].items():
            if eth_address == address_string:
                return eth_index

In [ ]:
meebits_string = "7bd29408f11d2bfc23c34f18275bbf23bb716bc7"
meebits_index = get_index(meebits_string)
meebits_bytes = bytes.fromhex(meebits_string)

In [ ]:
meebits_edges_file = Path("meebits_edges.pickle")
if meebits_edges_file.exists():
    with open(meebits_edges_file, "rb") as f:
        meebits_edges = pickle.load(f)
else:
    meebits_edges = [
        (u, v, t, data)
        for u, v, t, data
        in tqdm(GraphEdgeIterator("2021-10-01", "2022-07-01"), unit_scale=True)
        if meebits_index in (u, v)
    ]
    with meebits_edges_file.open("wb") as f:
        pickle.dump(meebits_edges, f)

meebits_star_graph = nx.MultiDiGraph()
meebits_star_graph.add_edges_from(meebits_edges)
print(meebits_star_graph)
print("In-Degree of meebits:", meebits_star_graph.in_degree(meebits_index))
print("Out-Degree of meebits:", meebits_star_graph.out_degree(meebits_index))

In [ ]:
def get_dwell_times(graph: nx.MultiDiGraph):
    # Initialize nested defaultdict to store dwell times.
    dwell_times = defaultdict(lambda: defaultdict(list))
    # Iterate over all nodes in the graph, treating each as a potential sender
    for sender in graph.nodes():
        # Look at all direct connections from this sender
        for intermediate, incoming_transactions in graph[sender].items():
            # Skip self-loops
            if sender == intermediate:
                continue
            # Sort the timestamps of incoming transactions to the intermediate node
            incoming_transactions = sorted(incoming_transactions.keys())
            # Pair each incoming transaction with the next one to form time windows
            incoming_transaction_pairs = zip(incoming_transactions, incoming_transactions[1:] + [None])
            for incoming_timestamp, next_incoming_time in incoming_transaction_pairs:
                # Examine all outgoing meebits_edges from the intermediate node
                for recipient, outgoing_transactions in graph[intermediate].items():
                    # Skip self-loops again
                    if intermediate == recipient:
                        continue
                    # Sort the timestamps of outgoing transactions from intermediate to recipient
                    outgoing_timestamps = sorted(outgoing_transactions.keys())
                    # Remove any outgoing timestamps that are before the incoming timestamp (strictly)
                    trim_before(outgoing_timestamps, lower_limit=incoming_timestamp, strict=True)
                    # If there is a subsequent incoming transaction, trim away timestamps after it (non-include_limit)
                    if next_incoming_time is not None:
                        trim_after(outgoing_timestamps, upper_limit=next_incoming_time, strict=False)
                    # For valid outgoing timestamps, calculate dwell time and add to result set
                    for outgoing_timestamp in outgoing_timestamps:
                        dwell_times[intermediate][sender].append(outgoing_timestamp - incoming_timestamp)
    return dwell_times


def get_burstiness(graph: nx.MultiDiGraph):
    dwell_times = get_dwell_times(graph)
    if len(dwell_times) == 0:
        return np.nan
    dwell_durations = []
    for intermediate, senders in dwell_times.items():
        for sender, durations in senders.items():
            # dwell_durations.extend(durations)
            dwell_durations.append(min(durations))
            # dwell_durations.append(np.mean(list(durations)))

    sigma, mu = np.std(dwell_durations), np.mean(dwell_durations)
    if sigma == mu:
        return 0.0
    return (sigma - mu) / (sigma + mu)


def draw_graph(graph):
    # Define the layout
    pos = nx.spring_layout(graph)

    # Draw nodes and labels
    nx.draw_networkx_nodes(graph, pos, node_size=500)
    nx.draw_networkx_labels(graph, pos)

    # Draw each edge with a different curvature
    edges = list(graph.edges(keys=True))
    for i, (u, v, k) in enumerate(edges):
        nx.draw_networkx_edges(
            graph,
            pos,
            edgelist=[(u, v)],
            connectionstyle=f'arc3,rad={(i - len(edges) / 2) * 0.2}',
            edge_color='black'
        )

    plt.axis('off')
    plt.show()

In [ ]:
component = components[3]
len(component)
# Sort the component_df
component_df = (
    raw_df.loc[list(component)].sort_values(
        by=["seed_begin", "seed_end"],
        ascending=[True, False]
    )
)
for bundled_cycle in component_df["bundled_cycle"]:
    # draw_graph(bundled_cycle)
    pass
plt.hist([graph.number_of_nodes() for graph in component_df["bundled_cycle"]])
plt.show()
plt.hist([graph.number_of_edges() for graph in component_df["bundled_cycle"]])
plt.show()

In [ ]:
raw_df["meebits"] = False
for u, v, t, data in tqdm(GraphEdgeIterator("2021-10-01", "2022-07-01"), unit_scale=True):
    if
        open_df = raw_df[~raw_df["meebits"]]
    open_df["meebits"] = open_df["vertices"].apply(lambda x: meebits_index in x)

    is_to_edge = u in merged_graph.nodes() and meebits_bytes == data["target_id"]
    is_from_edge = meebits_bytes == data["source_id"] and v in merged_graph.nodes()
    if is_to_edge or is_from_edge:
        merged_graph.add_edge(u, v, key=t, **data)
        meebits_edges.append((u, v, t, data))

print("Number of meebits_edges:", len(meebits_edges))

# Find the SCC that contains the target_node
for component in nx.strongly_connected_components(merged_graph):
    if meebits_index in component:
        print("Strongly Connected Component containing node", me, ":", component)
        break

In [ ]:
inside = 0
outside = 0

meebits_component = {meebits_index}
for component in nx.strongly_connected_components(merged_graph):
    if any(merged_graph.has_edge(component_vertex, meebits_index) for component_vertex in component):
        meebits_component |= component
print("Size of meebits_component:", len(meebits_component))

In [ ]:
burstiness = get_burstiness(merged_graph.subgraph(meebits_component))
print(f"Burstiness of meebits component: {burstiness:.4f}")

In [ ]:

raw_df["burstiness"] = raw_df["bundled_cycle"].apply(get_burstiness)
burstiness = raw_df["burstiness"].dropna()
burstiness = burstiness[burstiness > -1]
plt.violinplot(burstiness, showmeans=True, showmedians=True)
plt.show()